In [18]:
#安装环境依赖
#如要生成版本，不建议使用持久化安装，超过一定数量的存储空间，会生成不了版本
#安装paddleseg
! pip install -q paddleseg

In [19]:
# requirement
import os
import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import paddle
from paddle.nn import functional as F
import random
from paddle.io import Dataset
from visualdl import LogWriter
from paddle.vision.transforms import transforms as T
import paddle.nn as nn
import warnings
from PIL import Image as PilImage
import cv2
import paddle.fluid as fluid

In [23]:
#——————————Spatial Unit&Semantic Unit——————————
class Spatial_Unit(nn.Layer):
    def __init__(self, channels):
        super(Spatial_Unit, self).__init__()
        self.conv_l = nn.Sequential(
            nn.Conv2D(channels,channels, kernel_size=[1,3], stride=1, padding=1),
            nn.BatchNorm(channels),
            nn.ReLU(),
            nn.Conv2D(channels,channels, kernel_size=[3,1], stride=1, padding=0),
            nn.BatchNorm(channels),
            nn.ReLU()
        )
        self.conv_m = nn.Sequential(
            nn.Conv2D(channels,channels, kernel_size=[3,1], stride=1, padding=0),
            nn.BatchNorm(channels),
            nn.ReLU(),
            nn.Conv2D(channels,channels, kernel_size=[1,3], stride=1, padding=1),
            nn.BatchNorm(channels),
            nn.ReLU()
        )
        self.conv_r = nn.Sequential(
            nn.Conv2D(channels,channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm(channels),
            nn.ReLU()
        )
        self.convout = nn.Conv2D(channels,channels, kernel_size=1, stride=1, padding=0)

    def forward(self,x):
        xl = self.conv_l(x)
        xm = self.conv_m(x)
        xr = self.conv_r(x)

        out = paddle.add(xl,xm,xr)
        out = self.convout(out)

        return out

#——————————Semantic Unit——————————
class Semantic_Unit(nn.Layer):
    def __init__(self, channels, reduction=16):
        super(Semantic_Unit, self).__init__()
        self.conv1 = nn.Sequential(   
            nn.Conv2D(channels,channels, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm(channels),
            nn.ReLU())
        self.conv2 = nn.Sequential(   
            nn.Conv2D(channels*3,channels, kernel_size=3, stride=1, padding=1),         
            nn.BatchNorm(channels),
            nn.ReLU())
        self.convout = nn.Conv2D(channels,channels, kernel_size=1, stride=1, padding=0)

    def forward(self,x):
        x1 = self.conv1(x)
        x2 = self.conv1(x1)
        x3 = self.conv1(x2)
        x4 = paddle.concat([x1, x2, x3], axis=1)
        
        x4 = self.conv2(x4)
        x = self.convout(x)  
        out = x + x4 

        return out

In [21]:
#——————————————AGE——————————————
class AGE(nn.Layer):
    def __init__(self, ch_left, ch_down):
        super(AGE, self).__init__()
        self.Upsample = nn.Sequential(
            nn.Conv2D(ch_down,ch_left, kernel_size=1, stride=1, padding=0),
            nn.Upsample(scale_factor=2)
        )
        self.GAP = nn.AdaptiveAvgPool2D(output_size=1)
        self.conv = nn.Conv2D(3*ch_left,ch_left,kernel_size=1, stride=1, padding=0) 
        self.sig = nn.Sigmoid()
        self.conv11 = nn.Conv2D(2*ch_left,ch_left,kernel_size=1, stride=1, padding=0) 

    def forward(self, x, xd):
        xd = self.Upsample(xd)
        xm = paddle.concat([x,xd],axis=1)

        x1 = self.conv(xm)
        x2 = self.GAP(x1)
        x2 = self.sig(x2)

        x3 = x2*x
        x4 = x3+x
        xd = self.conv11(xd)
        x5 = paddle.concat([xd,x4],axis=1)

        return out

#————————————————AFM——————————————
class BasicConv(nn.Layer):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2D(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias_attr=bias)
        self.bn = nn.BatchNorm2D(out_planes) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x
        
class ChannelPool(nn.Layer):
    def forward(self, x):
        return paddle.concat( (paddle.max(x,1).unsqueeze(1), paddle.mean(x,1).unsqueeze(1)), axis=1 )

class AFM(nn.Layer):
    def __init__(self, channels):
        super(AFM, self).__init__()
        self.avgpool = nn.AvgPool2D(kernel_size=1, stride=1)
        self.conv2 = nn.Sequential(
            nn.Conv2D(channels,channels,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm(channels),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2D(channels*3,channels,kernel_size=1,stride=1,padding=0),
            nn.BatchNorm(channels),
            nn.ReLU()
        )
        self.sigmoid = nn.Sigmoid()
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size=3, stride=1, padding=(7-1) // 2, relu=False)
        
    def forward(self, xe, xp):
        xe1 = self.avgpool(xe)
        xe2 = fluid.layers.fc(xe1,size=1,num_flatten_dims=3)
        xe3 = self.sigmoid(xe2)
        xe4 = xe3*xe
        
        x_compress = self.compress(xp)
        x_out = self.spatial(x_compress)
        xp2 = F.sigmoid(x_out)
        xp3 = xp*xp2

        xm1 = xe*xp
        xm2 = self.conv2(xm1)

        xo = paddle.concat([xe4,xm2,xp3],axis=1)
        out = self.conv3(xo)
    
        return out

#——————————————GCF——————————————
class GCF(nn.Layer):
    def __init__(self, channels):
        super(GCF, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2D(channels,1,kernel_size=3,stride=1,padding=1,dilation=1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2D(channels,1,kernel_size=3,stride=1,padding=2,dilation=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2D(channels,1,kernel_size=3,stride=1,padding=4,dilation=4)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2D(channels,1,kernel_size=3,stride=1,padding=8,dilation=8)
        )
        self.conv11 = nn.Conv2D(4,1,kernel_size=1,stride=1,padding=0)
        self.conv33 = nn.Sequential(
            nn.Conv2D(2,1,kernel_size=7,stride=1,padding=3)
        )
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x1 = self.conv1(x)
        x1 = self.relu(x1)

        x2 = self.conv2(x)
        x2 = self.relu(x2)

        x3 = self.conv3(x)
        x3 = self.relu(x3)

        x4 = self.conv4(x)
        x4 = self.relu(x4)
        
        xm1 = paddle.concat([x1,x2,x3,x4],axis=1)
        
        xmm = paddle.max(xm1,1).unsqueeze(1)
        xma = paddle.mean(xm1,1).unsqueeze(1)
        xm2 = paddle.concat([xmm,xma],axis=1)

        xm3 = self.conv33(xm2)
        xm4 = self.sigmoid(xm3)
        out = xm4*x

        return out

#——————————————MAF——————————————
class MAF(nn.Layer):
    def __init__(self, channels):
        super(MAF, self).__init__()
        self.conv1 = nn.Conv2D(channels,1,kernel_size=1,stride=1,padding=0)
        self.conv2 = nn.Conv2D(3,1,kernel_size=3,stride=1,padding=1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        xm = paddle.max(x,1).unsqueeze(1)
        xa = paddle.mean(x,1).unsqueeze(1)
        xc = self.conv1(x)

        x1 = paddle.concat([xm,xa,xc],axis=1)

        x2 = self.conv2(x1)
        x3 = self.sigmoid(x2)

        out = x3*x
        out = out + x

        return out

#——————————————Decoder——————————————
class Decoder(nn.Layer):
    def __init__(self, ch_in, ch_out):
        super(Decoder, self).__init__()
        self.Cconv = nn.Sequential(
            nn.Conv2D(ch_in*2,ch_in,3,1,1),
            nn.BatchNorm(ch_in),
            nn.ReLU(),
            nn.Conv2D(ch_in,ch_in,3,1,1),
            nn.BatchNorm(ch_in),
            nn.ReLU()
        )
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2D(ch_in, ch_out, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm(ch_out),
            nn.ReLU()
        )
    def forward(self,xl,xd):
        x = paddle.concat([xl,xd],axis=1)
        out = self.Cconv(x)
        out = self.up(out)

        return out

#——————————————Attention_block——————————————
class Attention_block(nn.Layer):
    def __init__(self, F_g, F_l, F_int):
        super(Attention_block, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2D(F_g, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2D(F_l, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2D(F_int, 1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU()

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)

        return x * psi

class conv_block(nn.Layer):
    def __init__(self, ch_in, ch_out):
        super(conv_block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2D(ch_in, ch_out, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm(ch_out),
            nn.ReLU(),
            nn.Conv2D(ch_out, ch_out, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm(ch_out),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.conv(x)
        return x

In [ ]:
class ADE(nn.Layer):
    def __init__(self, img_ch=3, output_ch=3):
        super(ADE, self).__init__()
        self.inputconv = nn.Conv2D(3, 64, kernel_size=1, stride=1, padding=0)

        self.semantic1 = Semantic_Unit(channels=64)
        self.spatial1 = Spatial_Unit(channels=64)
        self.maxpool1 = nn.MaxPool2D(kernel_size=2,stride=2)
        self.afm1 = AFM(channels=128)

        self.semantic2 = Semantic_Unit(channels=128)
        self.spatial2 = Spatial_Unit(channels=128)
        self.maxpool2 = nn.MaxPool2D(kernel_size=2,stride=2)
        self.afm2 = AFM(channels=256)

        self.semantic3 = Semantic_Unit(channels=256)
        self.spatial3 = Spatial_Unit(channels=256)
        self.maxpool3 = nn.MaxPool2D(kernel_size=2,stride=2)
        self.afm3 = AFM(channels=512)

        self.semantic4 = Semantic_Unit(channels=512)
        self.spatial4 = Spatial_Unit(channels=512)
        self.maxpool4 = nn.MaxPool2D(kernel_size=2,stride=2)
        self.afm4 = AFM(channels=1024)

        self.gcf = GCF(channels=1024)

        self.de4 = Decoder(1024,512)
        self.age4 = AGE(ch_left=1024,ch_down=2048)
        self.maf4 = MAF(channels=1024)

        self.de3 = Decoder(512,256)
        self.age3 = AGE(ch_left=512,ch_down=1024)
        self.maf3 = MAF(channels=512)

        self.de2 = Decoder(256,128)
        self.age2 = AGE(ch_left=256,ch_down=512)
        self.maf2 = MAF(channels=256)

        self.de1 = Decoder(128,64)
        self.age1 = AGE(ch_left=128,ch_down=256)
        self.maf1 = MAF(channels=128)

        self.Att4 = Attention_block(F_g=1024, F_l=1024, F_int=512)
        self.Att3 = Attention_block(F_g=512, F_l=512, F_int=256)
        self.Att2 = Attention_block(F_g=256, F_l=256, F_int=128)
        self.Att1 = Attention_block(F_g=128, F_l=128, F_int=64)
        
        self.up = nn.Upsample(scale_factor=2)
        self.outconv = nn.Conv2D(64,3,kernel_size=1,stride=1,padding=0)
        
    def forward(self, x):
        x1 = self.inputconv(x)

        se1 = self.semantic1(x1)
        sp1 = self.spatial1(x1)
        afm1 = self.afm1(se1,sp1)
        mse1 = self.maxpool1(se1)
        msp1 = self.maxpool1(sp1)
        
        se2 = self.semantic2(mse1)
        sp2 = self.spatial2(msp1)
        afm2 = self.afm2(se2,sp2)
        mse2 = self.maxpool2(se2)
        msp2 = self.maxpool2(sp2)
        
        se3 = self.semantic3(mse2)
        sp3 = self.spatial3(msp2)
        afm3 = self.afm3(se3,sp3)
        mse3 = self.maxpool3(se3)
        msp3 = self.maxpool3(sp3)
        
        se4 = self.semantic4(mse3)
        sp4 = self.spatial4(msp3)
        afm4 = self.afm4(se4,sp4)
        mse4 = self.maxpool4(se4)
        msp4 = self.maxpool4(sp4)

        afm_d = self.maxpool4(afm4)
        gcf = self.gcf(afm_d)
        
        up = self.up(gcf)
        age4 = self.age4(afm4,gcf)
        maf4 = self.maf4(sge)
        att4 = self.Att4(maf4,up)
        de4 = self.de4(att4,up)
        
        age3 = self.age3(afm3,age4)
        maf3 = self.maf3(age3)
        att3 = self.Att3(maf3,de4)
        de3 = self.de3(att3,de4)
     
        age2 = self.age2(afm2,age3)
        maf2 = self.maf2(age2)
        att2 = self.Att2(maf2,de3)
        de2 = self.de2(att2,de3)
        
        age1 = self.age1(afm1,age2)
        maf1 = self.maf1(age1)
        att1 = self.Att1(maf1,de2)
        de1 = self.de1(att1,de2)

        out = self.outconv(de1)
        return out

IMAGE_SIZE = (224, 224)
network = ADE(img_ch=3, output_ch=3)
model = paddle.Model(network)
model.summary((-1, 3,) + IMAGE_SIZE)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 